In [2]:
import os
import cv2
import copy
import glob
import math
import imutils
import numpy as np

from sort.tools_sift import Stitcher,match_distance,EnhanceImage
from tools_xiaobo_cv import DrawKps,ImConcat
#from op_formats import NPMapOP25bToSMPL24
from sort.tools_sift import *
from sort.one_eurio_filter import OneEuroFilter
from sort.kalman import KalmanFilter
from sort.op_formats import NPMapOP25bToSMPL24
from sort.tools_bsp import BspFit,bsp_for_kp2d
from tools_xiaobo_cv import DrawKps,ImConcat

In [99]:
#可调整参数
bili=1
PointDistance = 5000
start_frame=101
workspace='D:/Sports/test/autotrack2/'
#bbox的2dpose
path_2dpose=workspace + 'pose/op25b_x2ds.npz'
#全景图
blackboard_path=workspace + 'panorama_v34.png'
#blackboard_path=workspace + 'test_panorama.png'
#bbox图片存放的文件夹
im_dir = workspace+'image/' 
#原视频
video_src=workspace+'/test.mp4'

#保存输出
save_npz = workspace+'2d_pose.npz'
save_guiji = workspace+'2d_track.npz'
show_video = workspace+'over_show.mp4'
save_video = workspace+'test_output.mp4'
#最后一帧得到全景带轨迹的图像
over_image = workspace+'all.png'
#轨迹图片相关
json_path = workspace+'jump.json'
model_image = workspace+'jump.png'

全景图与每个图片做匹配得到变换矩阵

In [40]:
#Main 全景图与每个图片做匹配得到变换矩阵
list_image_name = []
stitcher = Stitcher()
if __name__ == '__main__': 
    for im_name in glob.glob(im_dir+'*.png'):
        list_image_name.append(im_name)
    # load the two images and resize them to have a width of 400 pixels
    # (for faster processing)
    imageA = cv2.imread(blackboard_path)
    #imageA = EnhanceImage(imageA)
    #imageA = cv2.resize(imageA,None,fx=bili,fy=bili)
    for i in range(0,int(len(list_image_name))):
        #if i<100 and i>66:
        imageB = cv2.imread(list_image_name[i])
        #imageB = cv2.resize(imageB,None,fx=bili,fy=bili)
    # stitch the images together to create a panorama
    # showMatches=True 展示两幅图像特征的匹配,返回vis
        #imageB = EnhanceImage(imageB)
        print('\r %d'%i,end='')
        vis = stitcher.stitch(start_frame+i,[imageA, imageB], ratio=0.75,showMatches=True,reprojThresh=10.0)
        #cv2.imshow('result',cv2.resize(imageA,None,fx=0.6,fy=0.6))
        #cv2.waitKey(1)
        cv2.imshow('vis',cv2.resize(vis,None,fx=0.1,fy=0.1))
        cv2.waitKey(1)
cv2.destroyAllWindows()
np.save(workspace+'H.npy',stitcher.dict_H)

 168

局部2Dpose转移到全局pose

In [52]:
offline_H = np.load(workspace+'H.npy',allow_pickle=True).item()
list_kp2ds,list_2dpose_frame = get_kp2ds(list(offline_H.keys()),path_2dpose)

#print(list_kp2ds[0])
H = np.array(list(offline_H.values()))
list_H_frame=list(offline_H.keys())
#print(len(list_kp2ds))
np_kp2ds = np.array(list_kp2ds).reshape(np.array(list_kp2ds).shape[0],25,3)
print(H.shape,np_kp2ds.shape)

np_kp2ds_copy = copy.deepcopy(np_kp2ds)
for i in range(np_kp2ds.shape[0]):
    #print(np.array(np_kp2ds[i,:,0:2]).shape)
    pose=np.asarray([np_kp2ds[i]])
    b = np.where(pose==[0,0,0], 0, 1)[0,:,0]
    if np.sum(b)>=10:#去除错误pose的阈值
        np_kp2ds_transform = cv2.perspectiveTransform(np.array([np_kp2ds[i,:,0:2]]),H[list_H_frame.index(list_2dpose_frame[i])])
    #print(np_kp2ds_transform.shape)
        np_kp2ds_copy[i,:,0:2] = np_kp2ds_transform
    else:
        print(pose)
print(np_kp2ds_copy.shape)


(169, 3, 3) (169, 25, 3)
(169, 25, 3)


全局展示2dpose 保存展示视频
Save 2dpose guiji

In [53]:
#变量：
list_image_name,record_point,list_zero=[],[],[]
#轨迹
op25b={}
guiji={}
#from25btosmpl = NPMapOP25bToSMPL24(0.2)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(show_video,fourcc, 30.0, (1920,1080),True)

blackboard = cv2.imread(blackboard_path)
blackboard = cv2.resize(blackboard,None,fx=bili,fy=bili)
list_frame = list(offline_H.keys())
for im_name in glob.glob(im_dir+'*.png'):
    list_image_name.append(im_name)
#滤波
#one_euro_filter = OneEuroFilter(min_cutoff=0.03,beta=0.03,d_cutoff=0.02)
#kalman = KalmanFilter((list(np_kp2ds_copy[0])[0][0],list(np_kp2ds_copy[0])[0][1],0,0))
#pose = np.array([np_kp2ds_copy[1]])
#b1 = np.array([bili,bili,1])
#pose = pose*b1
#list_zero.append(smplpose[0,0:2])
#center = Op25bToTraj2d(copy.deepcopy(pose))
#center = one_euro_filter(0,center[-1,0:2])
#for i in range(20):
#    kalman.update(center[-1,0:2])

#for i in range(np_kp2ds_copy.shape[0]):
#    np_kp2ds_copy[i] = one_euro_filter(i,np_kp2ds_copy[i])
#one_euro_filter = OneEuroFilter()   
#print(np_kp2ds_copy.shape)
list_center=[]
smplpose = from25btosmpl(np_kp2ds_copy)
for i in range(smplpose.shape[0]):
    #pose = np.array([np_kp2ds_copy[i]])
    #center = Op25bToTraj2d(pose)
    center=smplpose[i][0][0:2]
    list_center.append([center])
np_center = np.array(list_center)#原始数据
print(np_center.shape)
bsp_kp2ds = bsp_for_kp2d(copy.deepcopy(np_center),alpha=0.0001)#BSP数据
print(bsp_kp2ds.shape)
#old_point=[]
for i in range(0,np_kp2ds_copy.shape[0]):
    #print(np.array([np_kp2ds_copy[i]]))
#     im =cv2.imread(list_image_name[list(offline_H.keys())[i]])
#     im = cv2.resize(im,None,fx=bili,fy=bili)
    pose = np.array([np_kp2ds_copy[i]])
    b1 = np.array([bili,bili,1])
    pose = pose*b1
    #smplpose = from25btosmpl(pose)
    #list_zero.append(smplpose[0,0:2])
    
    center = check_enter(bsp_kp2ds[i],PointDistance)
    #print(center)
    #print(center.shape)
    #center = one_euro_filter(i,center)
    #print(center.shape)
    #print(center[0])
    #print((list(np_kp2ds_copy[i][0])[0],list(np_kp2ds_copy[i][0])[1]))
    #print(center)
    #center = check_enter(center,PointDistance)
    #kalman.update(center)
    #x,y = kalman.get()
    list_zero.append((int(center[0]),int(center[1])))
    op25b[i]=np_kp2ds_copy[i]
    guiji[i]=((int(center[0]),int(center[1])))
    #print(center[0:2])
    #print(list_zero,list_zero[0][0][0])
    image = DrawKps(copy.deepcopy(blackboard),pose)
    if len(list_zero)>1:
        for i in range(len(list_zero)-1):
            list_=[]#[[x1,y1],[x2,y2]]
        #print(list_zero[i])
            cv2.line(image,list_zero[i],list_zero[i+1],(0,0,255),3)
    #im = DrawKps(im,pose)
    #cv2.imshow('0-0',cv2.resize(image,None,fx=0.4,fy=0.4))
    #cv2.imshow('1-1',cv2.resize(cv2.imread(list_image_name[i]),None,fx=0.4,fy=0.4))
    #im = ImConcat([cv2.resize(image,None,fx=0.3,fy=0.3),cv2.resize(im,None,fx=0.3,fy=0.3)])
    #print(im.shape)
    #print(list(stitcher.dict_H.keys())[i])
    cv2.imshow('0-0',cv2.resize(image,None,fx=0.1,fy=0.1))
    out.write(cv2.resize(image,(1920,1080)))
    cv2.waitKey(100)
cv2.imwrite(over_image,image)
out.release()
cv2.destroyAllWindows()
np.savez(save_npz,op25b = op25b)
np.savez(save_guiji,guiji=guiji)

(169, 1, 2)
(169, 2)


opencv 方法放入轨迹

轨迹动画生成

In [16]:
import json
from sort.tools_op import load_npz
#全景图的人体轨迹坐标
#save_npz = save_guiji
#全景图
src_image = blackboard_path
#op25b转smpl24
from25btosmpl = NPMapOP25bToSMPL24(0)

得到轨迹图片的三个点坐标

In [75]:
with open(json_path,'r') as load_f:
    load_dict = json.load(load_f)
list_point = []
for point in load_dict['shapes']:
    #print(point['label'])
    list_point.append((int(point['points'][0][0]),int(point['points'][0][1])))
print(list_point)
np_list_point=np.array(list_point)
print(np_list_point)

[(722, 46), (263, 412), (1171, 412)]
[[ 722   46]
 [ 263  412]
 [1171  412]]


找到轨迹跳跃的三个点top left right

In [131]:
frame_point = [53,92,21]
list_point_src=[]
src_im = cv2.imread(src_image)
im_jump = cv2.imread(model_image)
src_pose = np.load(save_guiji,allow_pickle=True)['guiji'].item()
for frame in frame_point:
    point = src_pose[frame]
    #print(src_pose[frame].shape)
    #pose = from25btosmpl(np.array([src_pose[frame]]))[0][0]
    #center = Op25bToTraj2d((np.array([src_pose[frame]])))
    #print(center)
    list_point_src.append((float(point[0]),float(point[1])))
    #print(np.array([src_pose[frame]]).shape)
print(list_point_src)
np_list_point_src = np.array(list_point_src)
print(np_list_point_src.shape)

[(1807.0, 673.0), (1357.0, 982.0), (2275.0, 1015.0)]
(3, 2)


数据图像旋转 保存

In [146]:
center_point = [(list_point_src[1][0]+list_point_src[2][0])/2,(list_point_src[1][1]+list_point_src[2][1])/2]
size_H,size_W = list_point_src[2][1] - list_point_src[1][1],list_point_src[2][0] - list_point_src[1][0]
angle = math.atan(size_W/size_H)
#angle = math.atan(size_W/size_H)
if size_H<0:
    angle =  angle*(180/math.pi)
    angle+=90
else:
    angle =  angle*(180/math.pi)
    angle-=90                    
print(angle)
im_jump = cv2.imread(model_image)
H,W = abs(list_point_src[0][1]-list_point_src[1][1]),abs(size_W)
H_src,W_src = abs(list_point[0][1] - list_point[1][1]),abs(list_point[1][0] - list_point[2][0])
fx = W/W_src
fy = H/H_src
print(fx,fy)
#print(np_list_point)
np_list_point[:,0]=np_list_point[:,0]*fx
#print(np_list_point)
np_list_point[:,1]=np_list_point[:,1]*fy
#print(np_list_point)
im_jump = cv2.resize(im_jump, None,fx=fx,fy=fy)
#for i in range(np_list_point.shape[0]):
#    cv2.circle(im_jump, (int(np_list_point[i][0]),int(np_list_point[i][1])), point_size, point_color, thickness)
dst  = my_rotate(copy.deepcopy(im_jump),center_point,angle,(255,255,255))
#dst = im_jump
print(im_jump.shape)
#cv2.imshow('dst',dst)
cv2.imwrite(save_npz.replace('.npz','_dst.png'),dst) 
dst_zeros=(int(np_list_point_src[0][0]-np_list_point[0][0]),int(np_list_point_src[0][1]-np_list_point[0][1]))
print(dst_zeros)

-2.0587657049905204
1.0110132158590308 0.8442622950819673
(474, 1434, 3)
(1014, 665)


In [151]:
#测试将全景图的三个点不断的还原到原始图中
blackboard = cv2.imread(blackboard_path)
list_image_name=[]
offline_H = np.load(workspace+'/H.npy',allow_pickle=True).item()
list_kp2ds,list_2dpose_frame = get_kp2ds(list(offline_H.keys()),path_2dpose)
print(len(list_kp2ds))
H = np.array(list(offline_H.values()))
list_H_frame=list(offline_H.keys())

point_size = 1
point_color = (0, 0, 255) # BGR
thickness = 8 # 可以为 0 、4、8
#video
video_src_cap = cv2.VideoCapture(video_src)
fps = video_src_cap.get(5)
w,h = int(video_src_cap.get(3)),int(video_src_cap.get(4))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(save_video,fourcc, fps, (w,h),True)

#
np_point_src = np.array(list_point_src)
real_point=np.zeros(np_point_src.shape)
for im_name in glob.glob(im_dir+'*.png'):
    list_image_name.append(im_name)
#st_point_minx,dst_point_maxx,dst_point_maxy,dst_point_miny    
blackboard_src = np.zeros(np.multiply(blackboard.shape,[2,2,1]),dtype='uint8')
borad_add_x,borad_add_y=int(blackboard.shape[0]/4),int(blackboard.shape[1]/4)
#放入dst
dst = cv2.imread(save_npz.replace('.npz','_dst.png')) 
blackboard_dst = np.ones(blackboard.shape,dtype='uint8')
blackboard_dst[dst_zeros[1]:dst_zeros[1]+dst.shape[0],dst_zeros[0]:dst_zeros[0]+dst.shape[1]]=dst
#print(len(list_image_name))
for i in range(len(list_image_name)):
    
    im = cv2.imread(list_image_name[i])
    A = np.matrix(H[list_H_frame.index(list_2dpose_frame[i])])
    
    #print(list_2dpose_frame[i])
    #print(blackboard.shape)
    #np_kp2ds_transform = cv2.perspectiveTransform(np.array([[[0.0,0.0]]]),H[list_H_frame.index(list_2dpose_frame[i])])[0][0]
    
    
    #cv2.circle(blackboard, (int(np_kp2ds_transform[0][0][0]),int(np_kp2ds_transform[0][0][1])), point_size, point_color, thickness)
    dst_out = cv2.warpPerspective(copy.deepcopy(blackboard_dst),A.I,(im.shape[1],im.shape[0]))
    #print(im.shape)
    cv2.imshow('dst_out',cv2.resize(dst_out,None,fx=0.5,fy=0.5))
    #dst_out = cv2.resize(dst_out,None,fx=2,fy=2)
    #print(dst_out.shape,blackboard.shape)
    #print(src_out.shape,blackboard.shape)
    borad_over = cv2.addWeighted(dst_out, 0.3, im, 1.3, 0)
    #src_im = borad_over[int(np_kp2ds_transform[1]):int(np_kp2ds_transform[1])+h,int(np_kp2ds_transform[0]):int(np_kp2ds_transform[0])+w]
    #cv2.imshow('blackboard',borad_over)
#     for num in range(np_point_src.shape[0]):
#         #print(np_point_src)
#         #print(np.array([[np_point_src[num]]]).shape)
#         #print(A.I.shape)
#         src_point = np.array([[np_point_src[num]]])
#         np_point_src_transform = cv2.perspectiveTransform(src_point,A.I)
#         #real_point[num] = np_point_src_transform[0][0]
#         cv2.circle(im, (int(np_point_src_transform[0][0][0]),int(np_point_src_transform[0][0][1])), point_size, point_color, thickness)
    out.write(borad_over)
    cv2.imshow('0-0',cv2.resize(borad_over,None,fx=0.5,fy=0.5))
    cv2.waitKey(1) 
out.release()
cv2.destroyAllWindows()

169


In [12]:
# #测试在全景图中不断变换图片所得到的视频
# offline_H = np.load(workspace+'/H.npy',allow_pickle=True).item()
# fourcc = cv2.VideoWriter_fourcc('H', '2', '6', '4')
# out = cv2.VideoWriter('D:/Sports/test/autotrack2/test_src_in_all.mp4',fourcc, 30, (4774,1620),True)
# H = np.array(list(offline_H.values()))
# list_image_name=[]
# for im_name in glob.glob(im_dir+'*.png'):
#     list_image_name.append(im_name)
# for i in range(H.shape[0]):
#     test_im=cv2.imread(list_image_name[i])
#     test_out = cv2.warpPerspective(test_im,H[i],(4774,1620))
#     out.write(test_out)
#     cv2.imshow('0-0',cv2.resize(test_out,None,fx=0.3,fy=0.3))
#     cv2.waitKey(100)
# out.release()
# cv2.destroyAllWindows()

In [91]:
#history

Unite@dafu

In [33]:
# #计算loss
# #np_center = np_center.reshape(np_center.shape[0],2)
# #print(bsp_kp2ds.shape)
# #loss_2dpose = np_center-bsp_kp2ds 


# src_im = cv2.imread(src_image)
# dst = cv2.imread(save_npz.replace('.npz','_dst.png'))
# #center_zero,center_end = (997,633),(2409,1203)
# center_zero = (int(list_point_src[0][0]-list_point[0][0]*fx), int(list_point_src[0][1]-list_point[0][1]*fy))
# point_size = 2
# point_color = (0, 0, 255) # BGR
# thickness = 4 # 可以为 0 、4、8
# offline_H = np.load(workspace+'/H.npy',allow_pickle=True).item()

# list_kp2ds,list_2dpose_frame = get_kp2ds(list(offline_H.keys()),path_2dpose)

# #print(list_2dpose_frame)
# #print(list_kp2ds[0])
# H = np.array(list(offline_H.values()))
# list_H_frame=list(offline_H.keys())
# #print(len(list_kp2ds))
# np_kp2ds = np.array(list_kp2ds).reshape(np.array(list_kp2ds).shape[0],25,3)
# np_zeros = np.zeros((H.shape[0],2))
# # A_zeros = np.zeros((H.shape[0],2))
# #print(np_zeros)
# print(np_zeros.shape,H.shape,np_kp2ds.shape)
# for i in range(np_zeros.shape[0]-1):
#     #print(np.array([[np_zeros[i]]]).shape)
#     np_kp2ds_transform = cv2.perspectiveTransform(np.array([[np_zeros[i]]]),H[list_H_frame.index(list_2dpose_frame[i])])
#     #print(np_kp2ds_transform.shape)
#     np_zeros[i]=np_kp2ds_transform[0][0]
# np_zeros = bsp_for_kp2d(copy.deepcopy(np_zeros).reshape(np_zeros.shape[0],1,2),alpha=0.0001)#BSP数据
# np_zeros = np_zeros.reshape(np_zeros.shape[0],2)
# #print(np_zeros.shape)
# balcekborad = cv2.imread(blackboard_path)
# borad = np.zeros((balcekborad.shape[0]*2,balcekborad.shape[1]*2,balcekborad.shape[2]),dtype=np.uint8)
# fourcc = cv2.VideoWriter_fourcc(*'XVID')
# out = cv2.VideoWriter(show_video.replace('.mp4','_complex.mp4'),fourcc, 30.0, (borad.shape[1],borad.shape[0]),True)
# out_src = cv2.VideoWriter(show_video.replace('.mp4','_src.mp4'),fourcc, 30.0, (1920,1080),True)
# print(balcekborad.shape[1]*2,balcekborad.shape[0]*2)
# print(borad.shape)
# for im_name in glob.glob(im_dir+'*.png'):
#     list_image_name.append(im_name)
    
# # for i in range(np_zeros.shape[0]-1):
# #     A_zeros[i][0]=center_zero[0] - (np_zeros[i][0]-np_zeros[0][0])
# #     A_zeros[i][1]=center_zero[1] - (np_zeros[i][1]-np_zeros[0][1])
# im_jump = cv2.imread(model_image)
# #center_zero = (center_zero[0]+int(borad.shape[1]/4),center_zero[1]+int(borad.shape[0]/4))
# #center_end = (center_end[0]+int(borad.shape[1]/4),center_end[1]+int(borad.shape[0]/4))


# #轨迹优化
# #print('kalman',np_zeros[0])
# # kalman = KalmanFilter((np_zeros[0][0],np_zeros[0][1],0,0))
# # for i in range(20):
# #     kalman.update(np_zeros[0])   
# # one_euro_filter = OneEuroFilter()
# list_zero=[]
# for i in range(np_zeros.shape[0]-1):
#     #np_zeros[i] = one_euro_filter(i,np_zeros[i])
#     #kalman.update(np_zeros[i])
#     #x,y = kalman.get()
#     #np_zeros[i]=(x,y)
#     im = cv2.imread(list_image_name[i])
# #     dst = cv2.imread(save_npz.replace('.npz','_dst.png'))
# #     A = np.matrix(H[i])
# #     dst = cv2.warpPerspective(dst,A.I,(dst.shape[1],dst.shape[0]))
# #     cv2.imshow('borad_dst',cv2.resize(dst,None,fx=0.5,fy=0.5))
# #     cv2.waitKey(1)
#     #print(np_zeros[0])
#     #print(0+int(np_zeros[0][1]),im.shape[0],borad.shape[1]-im.shape[1]+int(np_zeros[0][0]),borad.shape[1])
#     #print(im.shape)
#     borad_dst=np.zeros((balcekborad.shape[0]*2,balcekborad.shape[1]*2,balcekborad.shape[2]),dtype=np.uint8)
#     borad_add_x,borad_add_y=int(borad.shape[0]/4),int(borad.shape[1]/4)
# #     A = np.matrix(H[i])
# #     borad_dst = cv2.warpPerspective(borad_dst,A.I,(borad_dst.shape[1],borad_dst.shape[0]))
#     borad[int(np_zeros[i][1])+borad_add_x:im.shape[0]+int(np_zeros[i][1])+borad_add_x,int(np_zeros[i][0])+borad_add_y:im.shape[1]+int(np_zeros[i][0])+borad_add_y]=im
#     #borad_dst[center_zero[1]+borad_add_x:center_zero[1]+dst.shape[0]+borad_add_x,center_zero[0]+borad_add_y::center_zero[0]+dst.shape[1]]=dst
#     #borad_bask[int(np_zeros[i][1])+borad_add_x:im.shape[0]+int(np_zeros[i][1])+borad_add_x,int(np_zeros[i][0])+borad_add_y:im.shape[1]+int(np_zeros[i][0])+borad_add_y]
#     #rint(H[i].shape)
#     #A = np.matrix(H[i])
#     #borad_dst = cv2.warpPerspective(borad_dst,A.I,(borad_dst.shape[1],borad_dst.shape[0]))
#     #cv2.imshow('borad_dst',cv2.resize(borad_dst,None,fx=0.2,fy=0.2))
#     borad_over = cv2.addWeighted(borad, 1, borad_dst, 0.2, 0)
#     im_src = borad_over[int(np_zeros[i][1])+borad_add_x:im.shape[0]+int(np_zeros[i][1])+borad_add_x,int(np_zeros[i][0])+borad_add_y:im.shape[1]+int(np_zeros[i][0])+borad_add_y]
#     #画pose零点轨迹
#     #center = check_enter(bsp_kp2ds[i],PointDistance)
# #     list_zero.append((int(bsp_kp2ds[i][0]),int(bsp_kp2ds[i][1])))
# #     if len(list_zero)>1:
# #         for j in range(len(list_zero)-1):
# #             cv2.line(borad_over,(list_zero[j][0]+borad_add_y,list_zero[j][1]+borad_add_x),(list_zero[j+1][0]+borad_add_y,list_zero[j+1][1]+borad_add_x),(0,0,255),3)
#     #ret,im = cap.read()
#     #borad[0:im.shape[0],(borad.shape[1]-im.shape[1]):borad.shape[1]]=im
#     #print(im.shape,im_jump.shape)
#     #im_jump_resize = im_jump
#     #im_jump_resize = cv2.resize(copy.deepcopy(im_jump),None,fx=(im.shape[0]/balcekborad.shape[0]),fy=(im.shape[1]/balcekborad.shape[1]))
#     #print(im_jump.shape)
#     #print(int(np_zeros[i][1]),int(np_zeros[i][1]+im_jump.shape[1]),int(np_zeros[i][0]),int(np_zeros[i][0]+im_jump.shape[0]))
#     #if int(np_zeros[i][0]+im_jump_resize.shape[1])<np_zeros[i].shape[0] and int(np_zeros[i][1]+im_jump_resize.shape[0])<np_zeros[i].shape[1]:
#     #borad[int(np_zeros[i][1]):int(np_zeros[i][1]+im_jump_resize.shape[0]),int(np_zeros[i][0]):int(np_zeros[i][0]+im_jump_resize.shape[1])]=im_jump_resize
#     #print()
#     print('\r %d'%i,end='')
#     #print(np_zeros[i])
#     #cv2.circle(borad, (int(np_zeros[i][0]),int(np_zeros[i][1])), point_size, point_color, thickness)
#     #cv2.rectangle(borad, center_zero, center_end, (0, 255, 0), 3,cv2.LINE_4)
#     #cv2.circle(borad, center_zero, 10, (255,255,255), thickness)
#     #print(borad.shape)
#     out.write(borad_over)
#     out_src.write(im_src)
#     cv2.imshow('borad',cv2.resize(borad_over,None,fx=0.2,fy=0.2))
#     cv2.waitKey(1)
# out.release()
# out_src.release()
# # cv2.imwrite('D:/Sports/test/autotrack2/board.png',borad_over)
# # #cv2.waitKey(0)
# # # np.savetxt('D:/Sports/test/autotrack/zeros_guiji.txt',np_zeros)
# # np.savetxt('D:/Sports/test/autotrack2/board.txt',center_zero)

# # np.savetxt('D:/Sports/test/autotrack2/zeros_guiji_relative.txt',np_zeros)
# #print(np_zeros)
# cv2.destroyAllWindows()

(169, 2) (169, 3, 3) (169, 25, 3)
(1, 1, 2)
0 0
(1, 1, 2)
1 1
(1, 1, 2)
2 2
(1, 1, 2)
3 3
(1, 1, 2)
4 4
(1, 1, 2)
5 5
(1, 1, 2)
6 6
(1, 1, 2)
7 7
(1, 1, 2)
8 8
(1, 1, 2)
9 9
(1, 1, 2)
10 10
(1, 1, 2)
11 11
(1, 1, 2)
12 12
(1, 1, 2)
13 13
(1, 1, 2)
14 14
(1, 1, 2)
15 15
(1, 1, 2)
16 16
(1, 1, 2)
17 17
(1, 1, 2)
18 18
(1, 1, 2)
19 19
(1, 1, 2)
20 20
(1, 1, 2)
21 21
(1, 1, 2)
22 22
(1, 1, 2)
23 23
(1, 1, 2)
24 24
(1, 1, 2)
25 25
(1, 1, 2)
26 26
(1, 1, 2)
27 27
(1, 1, 2)
28 28
(1, 1, 2)
29 29
(1, 1, 2)
30 30
(1, 1, 2)
31 31
(1, 1, 2)
32 32
(1, 1, 2)
33 33
(1, 1, 2)
34 34
(1, 1, 2)
35 35
(1, 1, 2)
36 36
(1, 1, 2)
37 37
(1, 1, 2)
38 38
(1, 1, 2)
39 39
(1, 1, 2)
40 40
(1, 1, 2)
41 41
(1, 1, 2)
42 42
(1, 1, 2)
43 43
(1, 1, 2)
44 44
(1, 1, 2)
45 45
(1, 1, 2)
46 46
(1, 1, 2)
47 47
(1, 1, 2)
48 48
(1, 1, 2)
49 49
(1, 1, 2)
50 50
(1, 1, 2)
51 51
(1, 1, 2)
52 52
(1, 1, 2)
53 53
(1, 1, 2)
54 54
(1, 1, 2)
55 55
(1, 1, 2)
56 56
(1, 1, 2)
57 57
(1, 1, 2)
58 58
(1, 1, 2)
59 59
(1, 1, 2)
60 60
(1, 1, 2)


KeyboardInterrupt: 

得到带轨迹合成的视频，和输出的pose胡，画出最后需要的轨迹图

In [18]:
from sort.tools_op import load_npz

path_2dpose = './test/autotrack2/over_show_complex_2dpose.npz'
cap = cv2.VideoCapture('D:/Sports/test/autotrack2/over_show_complex.mp4')
op25b_2dpose = load_npz(path_2dpose)
root_guiji = Op25bToTraj2d(op25b_2dpose)[:,0:2]
print(root_guiji.shape)
root_guiji = root_guiji.reshape(root_guiji.shape[0],1,2)
bsp_root_guiji = bsp_for_kp2d(copy.deepcopy(root_guiji),alpha=0.0001)#BSP数据
W,H = int(cap.get(3)),int(cap.get(4))
count = int(cap.get(7))
fps = cap.get(5)
fourcc = cv2.VideoWriter_fourcc('H', '2', '6', '4')
out = cv2.VideoWriter('D:/Sports/test/autotrack2/over_show_complex_guiji.mp4',fourcc, fps, (W,H),True)
count = int(cap.get(7))
list_zero=[]
for i in range(bsp_root_guiji.shape[0]):
    ret,im = cap.read()
    list_zero.append((int(bsp_root_guiji[i][0]),int(bsp_root_guiji[i][1])))
    if len(list_zero)>1:
        for i in range(len(list_zero)-1):
            cv2.line(im,list_zero[i],list_zero[i+1],(0,0,255),3)
    cv2.imshow('0-0',cv2.resize(im,None,fx=0.2,fy=0.2))
    out.write(im)
    cv2.waitKey(1)
cv2.imwrite('D:/Sports/test/autotrack2/over_show_complex_guiji.png',im)
out.release()     
cap.release()
cv2.destroyAllWindows()

(166, 2)
